In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/GOVHACK

/content/drive/MyDrive/GOVHACK


In [ ]:
ls

 codes/
'Combined datasets used for the poewer BI Dashboard'/
'DATA SET FOR Delivering the 20-Minute Neighbourhood Plan'/
'DATA SET FOR Enabling Better Community Housing and Infrastructure Planning'/
'Gov hack 25.docx'
 no_rows_found.png
'what to do.docx'


In [ ]:
!pip -q install folium ipywidgets pandas requests shapely googlemaps
!pip -q install geopandas

from google.colab import output
output.enable_custom_widget_manager()


In [42]:
from dataclasses import dataclass
from typing import Optional, Dict, List
from pathlib import Path
import json, time, re, math, requests
import pandas as pd
import numpy as np
import folium
from folium import Map, LayerControl
from folium.features import DivIcon
from shapely.geometry import shape, Point
from branca.element import Template, MacroElement
import ipywidgets as W
from google.colab import userdata
from google.colab.userdata import SecretNotFoundError, NotebookAccessError
import os
from folium import Element

try:
    os.environ["GOOGLE_MAPS_API_KEY"] = userdata.get("GOOGLE_MAPS_API_KEY")
    GOOGLE_KEY = os.getenv("GOOGLE_MAPS_API_KEY")

    print("Loaded Google key")
except (SecretNotFoundError, NotebookAccessError):
    print("Secret not found; set it in the Secrets panel.")

Loaded Google key


In [49]:
# ---- FILE PATHS (edit to your Drive paths) ----
CSV_SCHOOLS_UNDER   = Path("DATA SET FOR Delivering the 20-Minute Neighbourhood Plan/Victorian School Building Authority/vic_school_buildings_2530_with_coords.csv")
CSV_SCHOOLS_EXIST   = Path("DATA SET FOR Delivering the 20-Minute Neighbourhood Plan/School Locations/dv402-SchoolLocations2025.csv")
CSV_STOPS_POINTS    = Path("DATA SET FOR Delivering the 20-Minute Neighbourhood Plan/Victorian Public Transport Lines and Stops/public_transport_stops.geojson")
CSV_PERMITS         = Path("DATA SET FOR Enabling Better Community Housing and Infrastructure Planning/Vic Government Data - Building Permits/20251496-Rawdata-July-20251_with_coords.csv")

# ---- FIXED PALETTES ----
SCHOOL_PALETTE = {
    "Schools under construction": "#D32F2F",  # strong red
    "Existing schools":           "#000000",  # BLACK
}

TRANSPORT_PALETTE = {
    "METRO BUS":       "#00796B",  # deep teal
    "METRO TRAM":      "#FFB300",  # amber
    "METRO TRAIN":     "#3949AB",  # indigo
    "REGIONAL TRAIN":  "#1B5E20",  # dark green
    "V/LINE COACH":    "#6D4C41",  # brown
    "FERRY":           "#00BFA5",  # aqua
}

PERMIT_PALETTE = {
    "Commercial":          "#D81B60",  # magenta
    "Domestic":            "#43A047",  # green
    "Hospital/Healthcare": "#8E24AA",  # purple
    "Industrial":          "#F4511E",  # deep orange
    "Public Buildings":    "#00ACC1",  # cyan
    "Residential":         "#5D4037",  # dark brown (differs from V/LINE)
    "Retail":              "#C0CA33",  # olive
}

# Normalize palette keys to uppercase (so they match uppercased category values)
PERMIT_PALETTE    = {k.upper(): v for k, v in PERMIT_PALETTE.items()}
TRANSPORT_PALETTE = {k.upper(): v for k, v in TRANSPORT_PALETTE.items()}

# ---- LAYER CONFIG (only colours changed; rest unchanged) ----
@dataclass
class LayerSpec:
    name: str
    csv_path: Path
    lat_col: str
    lon_col: str
    popup_cols: List[str]
    color: str
    filterable: bool = True
    category_col: Optional[str] = None
    color_map: Optional[Dict[str, str]] = None

LAYERS: List[LayerSpec] = [
    LayerSpec(
        name="Schools under construction",
        csv_path=CSV_SCHOOLS_UNDER,
        lat_col="coordinates y", lon_col="coordinates x",
        popup_cols=["School name","Suburb","Project type","What's happening?"],
        color=SCHOOL_PALETTE["Schools under construction"],
    ),
    LayerSpec(
        name="Existing schools",
        csv_path=CSV_SCHOOLS_EXIST,
        lat_col="Y", lon_col="X",
        popup_cols=["School_Name","School_Type","Address","Postcode"],
        color=SCHOOL_PALETTE["Existing schools"],   # BLACK
    ),
    LayerSpec(
        name="Public transport stops",
        csv_path=CSV_STOPS_POINTS,
        lat_col="coordinates y", lon_col="coordinates x",
        popup_cols=["STOP_NAME","MODE","STOP_ID"],
        color="#00acc1",                    # fallback (unused when category_col present)
        category_col="MODE",
        color_map=TRANSPORT_PALETTE,
    ),
    LayerSpec(
        name="Building permits",
        csv_path=CSV_PERMITS,
        lat_col="coordinates y", lon_col="coordinates x",
        popup_cols=["BASIS_Building_Use","Site_street_name","site_town_suburb__c","site_postcode__c"],
        color="#7e57c2",                    # fallback (unused when category_col present)
        category_col="BASIS_Building_Use",
        color_map=PERMIT_PALETTE,
    ),
]


In [50]:
class SuburbLocator:
    """Geocode a suburb/postcode to a polygon (preferred) or viewport/centroid fallback."""
    def __init__(self, google_key: Optional[str] = None, polite_sleep=1.0):
        self.google = None
        if google_key:
            try:
                import googlemaps
                self.google = googlemaps.Client(key=google_key)
            except Exception:
                self.google = None
        self.sess = requests.Session()
        self.sess.headers.update({"User-Agent": "govhack-locator/1.0"})
        self.sleep = polite_sleep

    def geocode(self, query: str):
        """Return dict with:
           - 'center': (lat, lon)
           - 'polygon': shapely geometry or None
           - 'bbox': (min_lat, min_lon, max_lat, max_lon) if no polygon
        """
        q = query.strip()
        if not q: return None

        # 1) Try Nominatim polygon (best for suburb boundaries)
        try:
            r = self.sess.get(
                "https://nominatim.openstreetmap.org/search",
                params={"q": f"{q}, Victoria, Australia", "format": "geojson", "polygon_geojson": 1, "limit": 1, "countrycodes": "au"},
                timeout=25,
            )
            r.raise_for_status()
            data = r.json()
            if data.get("features"):
                feat = data["features"][0]
                geom = feat.get("geometry")
                centroid = feat.get("properties", {}).get("centroid")  # rarely present
                shp = shape(geom) if geom else None
                center = (shp.centroid.y, shp.centroid.x) if shp else None
                time.sleep(self.sleep)
                return {"center": center, "polygon": shp, "bbox": None}
            time.sleep(self.sleep)
        except Exception:
            pass

        # 2) Fallback: Google viewport (or Nominatim point) → bbox
        center, bbox = None, None

        # Google first if available
        if self.google is not None:
            try:
                res = self.google.geocode(f"{q}, Victoria, Australia", region="au", components={"country":"AU"})
                if res:
                    loc = res[0]["geometry"]["location"]
                    center = (loc["lat"], loc["lng"])
                    vp = res[0]["geometry"].get("viewport")
                    if vp:
                        ne = vp["northeast"]; sw = vp["southwest"]
                        bbox = (sw["lat"], sw["lng"], ne["lat"], ne["lng"])
                    return {"center": center, "polygon": None, "bbox": bbox}
            except Exception:
                pass

        # Nominatim point fallback
        try:
            r = self.sess.get(
                "https://nominatim.openstreetmap.org/search",
                params={"q": f"{q}, Victoria, Australia", "format": "json", "limit": 1, "countrycodes": "au"},
                timeout=20,
            )
            if r.ok:
                arr = r.json()
                if arr:
                    center = (float(arr[0]["lat"]), float(arr[0]["lon"]))
            time.sleep(self.sleep)
        except Exception:
            pass

        # No precise bbox; synthesize a small one around center (3km)
        if center:
            lat, lon = center
            # ~0.027 deg ≈ 3km in latitude; adjust lon by cos(lat)
            dlat = 0.027
            dlon = dlat / max(0.3, math.cos(math.radians(lat)))
            bbox = (lat - dlat, lon - dlon, lat + dlat, lon + dlon)
            return {"center": center, "polygon": None, "bbox": bbox}
        return None


class CombinedSuburbMap:
    def __init__(self, layer_specs: List[LayerSpec], center=(-37.8136, 144.9631), zoom=9):
        self.specs = layer_specs
        self.center = center
        self.zoom = zoom
        self.dfs: Dict[str, pd.DataFrame] = {}
        self._load_layers()

    def _load_layers(self):
       for spec in self.specs:
        suffix = spec.csv_path.suffix.lower()
        if suffix in (".geojson", ".json"):
            import geopandas as gpd
            gdf = gpd.read_file(spec.csv_path)
            gdf = gdf[gdf.geometry.notnull()]
            # keep only points; if lines/polygons exist, skip them
            if "geom_type" in gdf.columns:
                gdf = gdf[gdf.geom_type == "Point"]
            else:
                gdf = gdf[gdf.geometry.geom_type == "Point"]
            # ensure the expected columns exist
            df = pd.DataFrame(gdf.drop(columns=["geometry"]))
            df[spec.lon_col] = gdf.geometry.x
            df[spec.lat_col] = gdf.geometry.y
        else:
            df = pd.read_csv(spec.csv_path, sep=None, engine="python")

        df.columns = [c.strip() for c in df.columns]
        # coerce coords
        df[spec.lat_col] = pd.to_numeric(df[spec.lat_col], errors="coerce")
        df[spec.lon_col] = pd.to_numeric(df[spec.lon_col], errors="coerce")
        df = df.dropna(subset=[spec.lat_col, spec.lon_col]).copy()
        self.dfs[spec.name] = df

    @staticmethod
    def _within_bbox(df, lat_col, lon_col, bbox):
        min_lat, min_lon, max_lat, max_lon = bbox
        return (df[lat_col].between(min_lat, max_lat)) & (df[lon_col].between(min_lon, max_lon))

    @staticmethod
    def _within_polygon(df, lat_col, lon_col, poly):
        # vectorized-ish point-in-polygon
        return df.apply(lambda r: poly.contains(Point(float(r[lon_col]), float(r[lat_col]))), axis=1)

    def _popup_html(self, row, cols):
        parts = []
        for c in cols:
            if c in row and not pd.isna(row[c]):
                parts.append(f"<b>{c}</b>: {row[c]}")
        return "<br>".join(parts)

    def render(self, area, radius_km: Optional[float] = None) -> Map:
        m = folium.Map(
            location=self.center if not area else area["center"],
            zoom_start=self.zoom,
            tiles="CartoDB positron"
        )

        # Optional: draw suburb shape/box
        if area and area.get("polygon"):
            folium.GeoJson(area["polygon"], name="Selected suburb").add_to(m)
        elif area and area.get("bbox"):
            min_lat, min_lon, max_lat, max_lon = area["bbox"]
            folium.Rectangle(
                bounds=[(min_lat, min_lon), (max_lat, max_lon)],
                color="#444", fill=False, weight=1, dash_array="4"
            ).add_to(m)

        legend_rows: List[str] = []

        # Filter & add layers
        for spec in self.specs:
            df = self.dfs[spec.name]
            mask = pd.Series([True] * len(df))
            if area:
                if area.get("polygon") is not None:
                    mask = self._within_polygon(df, spec.lat_col, spec.lon_col, area["polygon"])
                elif area.get("bbox") is not None:
                    mask = self._within_bbox(df, spec.lat_col, spec.lon_col, area["bbox"])

                # optional radius clamp
                if radius_km is not None and area.get("center"):
                    lat0, lon0 = area["center"]
                    lat = df[spec.lat_col].to_numpy(dtype=float)
                    lon = df[spec.lon_col].to_numpy(dtype=float)
                    R = 6371.0
                    dlat = np.radians(lat - lat0)
                    dlon = np.radians(lon - lon0)
                    a = np.sin(dlat/2)**2 + np.cos(np.radians(lat))*np.cos(np.radians(lat0))*np.sin(dlon/2)**2
                    dist = 2*R*np.arcsin(np.sqrt(a))
                    mask = mask & (dist <= float(radius_km))

            sub = df[mask].copy()
            if sub.empty:
                continue

            # Decide colors: per-category if category_col set, else fixed layer color
            cmap = None
            cats_present: List[str] = []
            if spec.category_col:
                # use original (trimmed) strings as keys
                cats_present = (
                    sub[spec.category_col]
                    .dropna()
                    .astype(str)
                    .map(lambda s: s.strip())
                    .unique()
                    .tolist()
                )
                cats_present.sort()

                if cats_present:
                    if spec.color_map:
                        # normalize only for lookup into the fixed palette
                        def _norm(x: str) -> str:
                            return str(x).strip().upper()
                        # local cmap keyed by ORIGINAL category strings
                        cmap = {c: spec.color_map.get(_norm(c), spec.color) for c in cats_present}
                    else:
                        # auto palette if no fixed map provided
                        cmap = ColorPicker.for_categories(cats_present)

            fg = folium.FeatureGroup(name=spec.name, show=True)
            for _, r in sub.iterrows():
                color = spec.color
                if cmap:
                    key = str(r.get(spec.category_col, "")).strip()
                    color = cmap.get(key, color)
                folium.CircleMarker(
                    location=(float(r[spec.lat_col]), float(r[spec.lon_col])),
                    radius=5, weight=1, color=color, fill=True, fill_color=color, fill_opacity=0.9,
                    popup=folium.Popup(self._popup_html(r, spec.popup_cols), max_width=360),
                ).add_to(fg)
            fg.add_to(m)

            # Legend rows
            if cmap and cats_present:
                legend_rows.append(f"<div style='font-weight:600;margin:4px 0 2px'>{spec.name}</div>")
                for c in cats_present:
                    legend_rows.append(
                        f"<div style='display:flex;align-items:center;margin:2px 0'>"
                        f"<span style='width:12px;height:12px;background:{cmap[c]};"
                        f"border:1px solid #555;display:inline-block;margin-right:6px'></span>"
                        f"<span style='font-size:12px'>{c}</span></div>"
                    )
            else:
                legend_rows.append(
                    f"<div style='display:flex;align-items:center;margin:2px 0'>"
                    f"<span style='width:12px;height:12px;background:{spec.color};"
                    f"border:1px solid #555;display:inline-block;margin-right:6px'></span>"
                    f"<span style='font-size:12px'>{spec.name}</span></div>"
                )

        LayerControl(position="topright", collapsed=False).add_to(m)

        # Legend (bottom-right, on top of the map)
        legend_html = f"""
        <div style="
          position: fixed; bottom: 16px; right: 16px; z-index: 10000;
          background: white; border: 1px solid #aaa; border-radius: 6px;
          box-shadow: 0 1px 3px rgba(0,0,0,.2); padding: 8px 10px; font-size: 12px;">
          <div style="font-weight:600;margin-bottom:4px;">Legend</div>
          {''.join(legend_rows)}
        </div>
        """
        m.get_root().html.add_child(Element(legend_html))

        return m

# ---- UI wiring ----
locator = SuburbLocator(google_key=GOOGLE_KEY)
app = CombinedSuburbMap(LAYERS)

txt = W.Text(
    value="",
    placeholder="Type suburb name)",
    description="Suburb:",
    layout=W.Layout(width="500px"),
    style={"description_width":"60px"},
)
btn = W.Button(description="Go", button_style="primary")
out = W.Output()

def _search(_=None):
    with out:
        out.clear_output(wait=True)
        q = txt.value.strip()
        if not q:
            print("Enter a suburb or postcode."); return
        area = locator.geocode(q)
        if not area or not area.get("center"):
            print("Could not locate that area."); return
        m = app.render(area, radius_km=50)   # ← fixed radius
        display(m)


btn.on_click(_search)
display(W.HBox([txt, btn]))
display(out)



Output()